In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

# ※ naver Open Api를 활용하여 네이버 지식인 "전주 여행"과 "경주 여행"을 검색 -> 명사만 추출(re) -> 빈도분석(DataFrame), 시각화(워드클라우드)

# 1. 네이버 api를 활용하여 두 검색어 추출

In [2]:
import os
import sys
import urllib.request
client_id = "PEdh_oUh_6HvzCnKIEa7"
client_secret = "FPyGzkvQLD"
encText = urllib.parse.quote("경주 여행")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Mon, 30 Dec 2024 16:42:59 +0900",
	"total":876464,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"<b>경주여행<\/b> 대릉원 드론촬영하기! (경주시청허가必)",
			"link":"https:\/\/blog.naver.com\/cooooooook\/223658719098",
			"description":"사진 ⓒ쿠우욱 안녕하세요 여행블로거 쿠우욱입니다 지난 <b>경주여행<\/b>을 떠나기전 경주 유적지들의 드론촬영을 해보고 싶어서 미리 경주시 문화유산과에 촬영 허가를 받고 방문하였습니다 :D 경주대릉원  위치... ",
			"bloggername":"태리네 가족은 여행중",
			"bloggerlink":"blog.naver.com\/cooooooook",
			"postdate":"20241113"
		},
		{
			"title":"<b>경주여행<\/b> 경주힐튼 조식 포함 숙박후기 실내수영장",
			"link":"https:\/\/blog.naver.com\/kkuik20\/223702301723",
			"description":"<b>경주여행<\/b> 경주힐튼 조식 포함 숙박후기 실내수영장 연말 <b>경주 여행<\/b> 오랜만에 와이프와 함께 <b>경주 여행<\/b>을 하게 되었다. 사실 경주 자체가 목적이었다기보다는 와이프 출장건과 더불어 여행과 주변 지역에... ",
			"bloggername":"꾸익의 일상",
			"bloggerlink":"blog.naver.com\/kkuik20",
			"postdate":"20241222"
		},
		{
			"title":"<b>경주 여행<\/b>코스 루지월드에서 만난 보물",
			"link":"https:\/\/blog.naver.com\/dcndb\/223661667998",
			"description":"이번에 처음 간 루지월드지만 나중에 <b>경주여행<\

In [3]:
from requests import get
import time
import os
import json
from html import unescape # &lt; -> <, &gt;-> >
import pandas as pd

In [6]:
# 네이버 API 계정 정보
client_id = 'PEdh_oUh_6HvzCnKIEa7'
client_secret = 'FPyGzkvQLD'
queries = ['전주 여행', '경주 여행']
max_start = 5

In [13]:
query = "전주 여행"
start = 0
headers = {'X-Naver-Client-Id':client_id,
           'X-Naver-Client-Secret':client_secret}
url = f'https://openapi.naver.com/v1/search/kin.json?query={query}&display=100&start={start+1}'
response = get(url, headers=headers)
print(response.text[:500])

{
	"lastBuildDate":"Mon, 30 Dec 2024 17:28:04 +0900",
	"total":47106,
	"start":1,
	"display":100,
	"items":[
		{
			"title":"<b>전주여행<\/b> 대중교통이랑 렌트카중 어떤게 좋을까요?",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=9&dirId=90110&docId=477694429&qb=7KCE7KO8IOyXrO2WiQ==&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0",
			"description":"... <b>전주여행<\/b>때 움직이기 차가 편할지 대중교통이 편할지 알려주세요. 교통 상황이 어떤지 궁금합니다. <b>전주여행<\/b> 전주맛집 국내여행 전주달봉이 달봉티비 <b>전주여행<\/b>코스 전주가볼만한곳 여수여행 여수가볼만한곳 제주도여행... "
		},
		


In [14]:
# 데이터를 저장할 파일 생성 (naver_kin.txt)하고 title 저장
file_name = 'data/naver_kin.txt'
with open(file_name, 'w', encoding='utf-8') as file:
    file.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format('query', 
                            'no','title','link', 'description', 'total_text'))

In [16]:
elements = json.loads(response.text)['items']
elements[0]

{'title': '<b>전주여행</b> 대중교통이랑 렌트카중 어떤게 좋을까요?',
 'link': 'https://kin.naver.com/qna/detail.naver?d1id=9&dirId=90110&docId=477694429&qb=7KCE7KO8IOyXrO2WiQ==&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0',
 'description': '... <b>전주여행</b>때 움직이기 차가 편할지 대중교통이 편할지 알려주세요. 교통 상황이 어떤지 궁금합니다. <b>전주여행</b> 전주맛집 국내여행 전주달봉이 달봉티비 <b>전주여행</b>코스 전주가볼만한곳 여수여행 여수가볼만한곳 제주도여행... '}

In [22]:
for i, element in enumerate(elements):
    title = element['title'].replace('<b>', ' ').replace('</b>', ' ')
    link = element['link']
    description = element['description'].replace('<b>', ' ').replace('</b>', ' ')
    description = unescape(description) # html 특수문자 처리(&lt; -> <로 대체)
    # print(query, (start*100+i+1), title, link, description, title+' ' +description)
    with open(file_name, 'a', encoding='utf-8') as file:
        file.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(query, 
                                                     start*100+i+1,
                                                     title,
                                                     link,
                                                     description, 
                                                     title + ' ' +description))

In [23]:
df = pd.read_csv('data/naver_kin.txt', sep='\t')
df.head(2)

,query,no,title,link,description,total_text
0,전주 여행,1,전주여행 대중교통이랑 렌트카중 어떤게 좋을까요?,https://kin.naver.com/qna/detail.naver?d1id=9&...,... 전주여행 때 움직이기 차가 편할지 대중교통이 편할지 알려주세요. 교통 상황...,전주여행 대중교통이랑 렌트카중 어떤게 좋을까요? ... 전주여행 때 움직이기 ...
1,전주 여행,2,전주여행 갈려고하는데요!,https://kin.naver.com/qna/detail.naver?d1id=9&...,전주여행 을 갈려고하는데요!아는사람과 갈려고하는데 호텔은 좋은가격에 정했고~ 음....,전주여행 갈려고하는데요! 전주여행 을 갈려고하는데요!아는사람과 갈려고하는데 호텔...


# 내일(12.31)

In [ ]:
from requests import get
import time
import os
import json
from html import unescape # &lt; -> <, &gt;-> >
import pandas as pd

In [ ]:
# 네이버 API 계정 정보
client_id = 'PEdh_oUh_6HvzCnKIEa7'
client_secret = 'FPyGzkvQLD'
queries = ['전주 여행', '경주 여행']
max_start = 5

In [ ]:
def get_search_element_save(query, start):
    headers = {'X-Naver-Client-Id':client_id,
           'X-Naver-Client-Secret':client_secret}
    url = f'https://openapi.naver.com/v1/search/kin.json?query={query}&display=100&start={start+1}'
    response = get(url, headers=headers)
    elements = json.loads(response.text)['items']
    for i, element in enumerate(elements):
        title = element['title'].replace('<b>', ' ').replace('</b>', ' ')
        link = element['link']
        description = element['description'].replace('<b>', ' ').replace('</b>', ' ')
        description = unescape(description) # html 특수문자 처리(&lt; -> <로 대체)
        # print(query, (start*100+i+1), title, link, description, title+' ' +description)
        with open(file_name, 'a', encoding='utf-8') as file:
            file.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format(query, 
                                                         start*100+i+1,
                                                         title,
                                                         link,
                                                         description, 
                                                         title + ' ' +description))

In [ ]:
# 데이터를 저장할 파일 생성 (naver_kin.txt)하고 title 저장
file_name = 'data/naver_kin.txt'
with open(file_name, 'w', encoding='utf-8') as file:
    file.write('{}\t{}\t{}\t{}\t{}\t{}\n'.format('query', 
                            'no','title','link', 'description', 'total_text'))